In [102]:
import pandas as pd
from datetime import timedelta
# basic visualization package
import matplotlib.pyplot as plt
# advanced ploting
import seaborn as sns

# interactive visualization
import plotly.express as px
import plotly.graph_objs as go
# import plotly.figure_factory as ff
from plotly.subplots import make_subplots
# for offline ploting
from plotly.offline import plot, iplot, init_notebook_mode
init_notebook_mode(connected=True)

In [103]:
kg_df = pd.read_csv("archive/country_wise_latest.csv")
kg_df['Country'] =kg_df['Country/Region']  
who_df = pd.read_csv("archive/who.csv")


fil_kg=  kg_df[['Country','Confirmed', 'Deaths', 'Recovered', 'Active','1 week change',
                '1 week % increase']]
fil_who = who_df[['Country','Adult literacy rate (%)','Population in urban areas (%)',
                  'Population median age (years)','Population proportion over 60 (%)',
                  'Population proportion under 15 (%)','Health_expenditure_total',
                  'Surface_area','Total_income','Urban_population']]



df = fil_kg.merge(fil_who,how='outer',on="Country")
# df.dropna(inplace=True)

In [104]:
# columns & other Xforms
df['Recovered_Percentage'] = df['Recovered']/df['Confirmed']*100


In [157]:
def plot_map(df, col, pal,title=""):
    df = df[df[col]>0]

    if not title:
        title = col

    fig = px.choropleth(df, locations="Country", locationmode='country names', 
                  color=col, hover_name="Country", 
                  title=title, hover_data=[col], color_continuous_scale=pal)
    # fig.update_layout(coloraxis_showscale=False)
    fig.show()

def plot_hbar_wm(df,col, title=""  ):
    df = df.sort_values(col, ascending=True)

    if not title:
        title = col

    fig = px.bar(df,
                 x=col, y="Country", color='Country',  
                 text=col, orientation='h', width=700, 
                 color_discrete_sequence = px.colors.qualitative.Dark2)
    fig.update_layout(title=title, 
                      xaxis_title=col, yaxis_title="Country", 
                      yaxis_categoryorder = 'total ascending',
                      uniformtext_minsize=8, uniformtext_mode='hide')
    fig.show()

In [159]:
plot_map(df, 'Deaths', 'matter')

# Top 10 countries based on death-count

In [160]:
_ = df.sort_values('Deaths',ascending=False).head(10)[['Country','Deaths']]
# _.style.hide_index()
plot_hbar_wm(_,'Deaths')


In [114]:
plot_map(_, 'Deaths', 'matter','Top 10 countries based on death-count')

# Recovered Percentage

In [115]:
plot_map(df, 'Recovered_Percentage', 'matter')

## Top 20 countries based on Recovery rates are the ones with low surface area

In [162]:
_ = df.sort_values('Recovered_Percentage',ascending=False).dropna(subset=['Surface_area']).head(20)
_insight = f"Countries with Avg. Surface Area around {round(_['Surface_area'].mean())} m.sq had better recovery rates" 
plot_map(_, 'Recovered_Percentage', 'matter',_insight)

In [163]:
plot_hbar_wm(_,'Surface_area')

# Recovery based on Literacy Rate

In [176]:

top= df.sort_values(['Recovered_Percentage'],ascending=False).dropna(subset=['Adult literacy rate (%)']).head(10)
bot= df.sort_values(['Recovered_Percentage']).dropna(subset=['Adult literacy rate (%)']).head(10)

plot_map(df, 'Adult literacy rate (%)', 'matter')

In [174]:
_top_insight = f"Countries with most Recovered% had an Avg. Adult literacy rate of {round(top['Adult literacy rate (%)'].mean())}%"
_bot_insight = f"Countries with least Recovered% had an Avg. Adult literacy rate of {round(bot['Adult literacy rate (%)'].mean())}%"
plot_hbar_wm(top, 'Adult literacy rate (%)',_top_insight)
plot_hbar_wm(bot, 'Adult literacy rate (%)',_bot_insight)
